In [ ]:
!pip install faiss-cpu
#!pip install faiss-gpu

In [15]:
from dotenv import load_dotenv

import os
import numpy as np
import pickle
import json
import bs4
from bs4 import BeautifulSoup
import requests
import uuid
import nest_asyncio
from tqdm import tqdm
from operator import itemgetter
from datetime import datetime
from pydantic import BaseModel, Field
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from seaborn import load_dataset

from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache
from langchain_core.load import dumpd, dumps, load, loads
from langchain_huggingface import HuggingFaceEndpoint
from langchain_community.callbacks.manager import get_openai_callback
from langchain_community.chat_models import ChatOllama
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain_core.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.example_selectors import (
    SemanticSimilarityExampleSelector,
    MaxMarginalRelevanceExampleSelector,
)
from langchain_core.output_parsers import (
    StrOutputParser,
    PydanticOutputParser,
    CommaSeparatedListOutputParser,
    JsonOutputParser,
)
from langchain.output_parsers.structured import (
    ResponseSchema,
    StructuredOutputParser,
)
from langchain.output_parsers.pandas_dataframe import PandasDataFrameOutputParser
from langchain.output_parsers.datetime import DatetimeOutputParser
from langchain.output_parsers.fix import OutputFixingParser
from langchain.output_parsers.retry import RetryOutputParser
from langchain_core.runnables import (
    RunnableParallel,
    RunnablePassthrough,
    RunnableLambda,
    ConfigurableField,
)

from langchain_core.documents import Document
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import (
    UnstructuredPDFLoader,
    PyPDFium2Loader,
    PDFMinerLoader,
    PyPDFDirectoryLoader,
    PDFPlumberLoader,
    UnstructuredExcelLoader,
    DataFrameLoader,
    Docx2txtLoader,
    UnstructuredWordDocumentLoader,
    UnstructuredPowerPointLoader,
    WebBaseLoader,
    TextLoader,
    DirectoryLoader,
    JSONLoader,
    ArxivLoader,
)

from langchain_text_splitters import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
    Language,
    MarkdownHeaderTextSplitter,
    HTMLHeaderTextSplitter,
    RecursiveJsonSplitter,
)
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain.storage import (
    LocalFileStore,
    InMemoryByteStore,
    InMemoryStore,
)
from langchain.embeddings import CacheBackedEmbeddings

import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_chroma import Chroma

from langchain_teddynote.document_compressors import LLMChainExtractor
from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain.retrievers import (
    ContextualCompressionRetriever,
    BM25Retriever,
    EnsembleRetriever,
    ParentDocumentRetriever,
    TimeWeightedVectorStoreRetriever,
)
from langchain_community.document_transformers import LongContextReorder
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever

In [20]:
import pandas as pd

In [16]:
load_dotenv()

True

In [21]:
df = pd.read_csv('./data/아파트(전월세)_실거래가_2024.csv')

In [29]:
df.columns

Index(['NO', '시군구', '번지', '본번', '부번', '단지명', '전월세구분', '전용면적(㎡)', '계약년월', '계약일',
       '보증금(만원)', '월세금(만원)', '층', '건축년도', '도로명', '계약기간', '계약구분', '갱신요구권 사용',
       '종전계약 보증금(만원)', '종전계약 월세(만원)', '주택유형'],
      dtype='object')

In [30]:
df.filter(items=['시군구', '번지', '본번', '부번', '단지명', '전월세구분', '전용면적(㎡)', '계약년월', '계약일',
       '보증금(만원)'])

,시군구,번지,본번,부번,단지명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원)
0,서울특별시 용산구 한강로2가,426,426,0,용산베르디움프렌즈,월세,48.9018,202412,6,"19,323"
1,서울특별시 용산구 한강로2가,426,426,0,용산베르디움프렌즈,월세,49.9403,202412,6,"19,707"
2,서울특별시 용산구 한강로2가,426,426,0,용산베르디움프렌즈,월세,44.9009,202412,6,"17,729"
3,서울특별시 종로구 숭인동,202-3,202,3,종로중흥S클래스,월세,17.8110,202412,6,500
4,서울특별시 성동구 마장동,831,831,0,마장에스에이치빌,월세,39.8400,202412,6,"6,813"
...,...,...,...,...,...,...,...,...,...,...
212853,서울특별시 서대문구 홍제동,453,453,0,청구1차,월세,84.7800,202401,1,"29,160"
212854,서울특별시 은평구 대조동,197-1,197,1,유진아이디움,전세,14.0300,202401,1,"13,000"
212855,서울특별시 서대문구 홍은동,202-5,202,5,"현대(1,2차)",월세,84.7700,202401,1,0
212856,서울특별시 서대문구 북아현동,1009,1009,0,두산,전세,59.9600,202401,1,"40,000"


In [31]:
df.filter(items=['월세금(만원)', '층', '건축년도', '도로명', '계약기간', '계약구분', '갱신요구권 사용',
       '종전계약 보증금(만원)', '종전계약 월세(만원)', '주택유형'])

,월세금(만원),층,건축년도,도로명,계약기간,계약구분,갱신요구권 사용,종전계약 보증금(만원),종전계약 월세(만원),주택유형
0,34,15,2021,백범로99길 40,202503~202703,갱신,-,"19,323",33,아파트
1,35,7,2021,백범로99길 40,202503~202703,갱신,-,"19,707",34,아파트
2,32,9,2021,백범로99길 40,202503~202703,갱신,-,"17,729",31,아파트
3,63,5,2013,종로66길 28,202501~202601,갱신,-,500,60,아파트
4,3,7,2006,마조로15가길 23,-,신규,-,NaN,NaN,아파트
...,...,...,...,...,...,...,...,...,...,...
212853,83,9,1994,통일로 348,202401~202512,갱신,사용,"28,000",78,아파트
212854,0,9,2019,연서로28길 8,202401~202601,갱신,-,"13,000",0,아파트
212855,100,9,1988,연희로 377,202401~202403,신규,-,NaN,NaN,아파트
212856,0,12,1999,이화여대8길 62,202401~202501,갱신,-,"30,000",10,아파트


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212858 entries, 0 to 212857
Data columns (total 21 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   NO            212858 non-null  int64  
 1   시군구           212858 non-null  object 
 2   번지            212858 non-null  object 
 3   본번            212858 non-null  int64  
 4   부번            212858 non-null  int64  
 5   단지명           212858 non-null  object 
 6   전월세구분         212858 non-null  object 
 7   전용면적(㎡)       212858 non-null  float64
 8   계약년월          212858 non-null  int64  
 9   계약일           212858 non-null  int64  
 10  보증금(만원)       212858 non-null  object 
 11  월세금(만원)       212858 non-null  object 
 12  층             212858 non-null  int64  
 13  건축년도          212858 non-null  int64  
 14  도로명           212858 non-null  object 
 15  계약기간          212858 non-null  object 
 16  계약구분          212858 non-null  object 
 17  갱신요구권 사용      212858 non-null  object 
 18  종전계약

In [37]:
from geopy.geocoders import Nominatim

def geocoding(address):
    geolocoder = Nominatim(user_agent = 'South Korea', timeout=None)
    geo = geolocoder.geocode(address)
    crd = {"lat": str(geo.latitude), "lng": str(geo.longitude)}

    return crd

In [38]:
crd = geocoding('백범로99길 40')

In [39]:
print(crd['lat'])
print(crd['lng'])

37.53594655
126.97154493121805


In [42]:
!pip install folium

In [43]:
import folium

In [47]:
folium.Map(location=[crd['lat'], crd['lng']], zoom_start=15)

In [5]:
# CSVLoader 클래스를 사용하여 CSV 파일 로드
loader = CSVLoader(file_path='./data/TR222100060001_20230707.csv')  # CSV 파일 경로 지정

# CSV 파일에서 데이터 내용 로드
data = loader.load()  # 로드된 데이터가 data 변수에 저장

# 로드된 데이터 출력
data

[Document(metadata={'source': './data/TR222100060001_20230707.csv', 'row': 0}, page_content='sigungu_cd: 11110\nemdl_cd: 10700\ncll: 1\nmno: 0156\nsno: 0000\nadres: 서울특별시 종로구 적선동 156\nhus_tp: 오피스텔\ncomp_nm: 광화문 플래티넘\nbldg_year: 2003\nflr: 14\nxuar: 38.64\nctrt_yrmth: 202306\nctrt_day: 21\ntransct_type: 매매\ndlng_amount: 23000\ngrnte_amount: 0\nmthrnt_amount: 0\nnear_subw_nm: 경복궁(정부서울청사)역\nnear_subw_dist: 102.3'),
 Document(metadata={'source': './data/TR222100060001_20230707.csv', 'row': 1}, page_content='sigungu_cd: 11110\nemdl_cd: 11500\ncll: 1\nmno: 0009\nsno: 0000\nadres: 서울특별시 종로구 사직동 9\nhus_tp: 아파트\ncomp_nm: 광화문스페이스본(101동~105동)\nbldg_year: 2008\nflr: 13\nxuar: 108.55\nctrt_yrmth: 202306\nctrt_day: 17\ntransct_type: 월세\ndlng_amount: 0\ngrnte_amount: 60000\nmthrnt_amount: 120\nnear_subw_nm: 경복궁(정부서울청사)역\nnear_subw_dist: 236.2'),
 Document(metadata={'source': './data/TR222100060001_20230707.csv', 'row': 2}, page_content='sigungu_cd: 11110\nemdl_cd: 11500\ncll: 1\nmno: 0009\nsno: 0000\

In [8]:
data_1 = data[0].page_content.split('\n')

In [9]:
data_1

['sigungu_cd: 11110',
 'emdl_cd: 10700',
 'cll: 1',
 'mno: 0156',
 'sno: 0000',
 'adres: 서울특별시 종로구 적선동 156',
 'hus_tp: 오피스텔',
 'comp_nm: 광화문 플래티넘',
 'bldg_year: 2003',
 'flr: 14',
 'xuar: 38.64',
 'ctrt_yrmth: 202306',
 'ctrt_day: 21',
 'transct_type: 매매',
 'dlng_amount: 23000',
 'grnte_amount: 0',
 'mthrnt_amount: 0',
 'near_subw_nm: 경복궁(정부서울청사)역',
 'near_subw_dist: 102.3']

In [19]:
from langchain.chains import (
    LLMChain,
    SimpleSequentialChain,
    SequentialChain,
    RetrievalQA,
    RetrievalQAWithSourcesChain,
)

In [21]:
# data: SPRi AI brief
# pdf load
loader = PyPDFLoader('./data/SPRi AI Brief_10월호_산업동향_F.pdf')
documents = loader.load()

# splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50,
)
documents = text_splitter.split_documents(documents)

# to db
db = FAISS.from_documents(
    documents=documents,
    embedding=OpenAIEmbeddings(model='text-embedding-3-small')
)

In [22]:
documents

[Document(metadata={'source': './data/SPRi AI Brief_10월호_산업동향_F.pdf', 'page': 0}, page_content='2024 년 10월호'),
 Document(metadata={'source': './data/SPRi AI Brief_10월호_산업동향_F.pdf', 'page': 1}, page_content='2024 년 10월호\nⅠ. 인공지능 산업 동향 브리프\n 1. 정책/법제 \n   ▹ 미·영·EU, 법적 구속력 갖춘 유럽평의회의 AI 국제조약에 서명········································· 1\n   ▹ 미국 캘리포니아 주지사 , AI 규제법안 「SB1047 」에 거부권 행사······································ 2\n   ▹ 호주 의회, 동의 없는 딥페이크 음란물 공유를 처벌하는 법안 통과·································· 3'),
 Document(metadata={'source': './data/SPRi AI Brief_10월호_산업동향_F.pdf', 'page': 1}, page_content='▹ UN, ‘인류를 위한 AI 거버넌스 ’ 최종 보고서 발표····························································· 4\n \n 2. 기업/산업 \n   ▹ 앤스로픽과 오픈AI, 미국 AI 안전연구소와 모델 평가 합의················································ 5'),
 Document(metadata={'source': './data/SPRi AI Brief_10월호_산업동향_F.pdf', 'page': 1}, page_content='▹ 오픈AI, 추론에 특화된 AI 모델 ‘o1-프리뷰 ’ 출시······························································ 6

In [31]:
import pandas as pd

In [52]:
df = pd.read_csv('./data/아파트(전월세)_실거래가_2024.csv')

In [54]:
df.columns

Index(['NO', '시군구', '번지', '본번', '부번', '단지명', '전월세구분', '전용면적(㎡)', '계약년월', '계약일',
       '보증금(만원)', '월세금(만원)', '층', '건축년도', '도로명', '계약기간', '계약구분', '갱신요구권 사용',
       '종전계약 보증금(만원)', '종전계약 월세(만원)', '주택유형'],
      dtype='object')

In [60]:
df.filter(items=['시군구', '단지명', '전월세구분', '계약기간', '종전계약 보증금(만원)', '종전계약 월세(만원)'])

,시군구,단지명,전월세구분,계약기간,종전계약 보증금(만원),종전계약 월세(만원)
0,서울특별시 용산구 한강로2가,용산베르디움프렌즈,월세,202503~202703,"19,323",33
1,서울특별시 용산구 한강로2가,용산베르디움프렌즈,월세,202503~202703,"19,707",34
2,서울특별시 용산구 한강로2가,용산베르디움프렌즈,월세,202503~202703,"17,729",31
3,서울특별시 종로구 숭인동,종로중흥S클래스,월세,202501~202601,500,60
4,서울특별시 성동구 마장동,마장에스에이치빌,월세,-,NaN,NaN
...,...,...,...,...,...,...
212853,서울특별시 서대문구 홍제동,청구1차,월세,202401~202512,"28,000",78
212854,서울특별시 은평구 대조동,유진아이디움,전세,202401~202601,"13,000",0
212855,서울특별시 서대문구 홍은동,"현대(1,2차)",월세,202401~202403,NaN,NaN
212856,서울특별시 서대문구 북아현동,두산,전세,202401~202501,"30,000",10


In [80]:
df['종전계약 보증금(만원)']

0         19,323
1         19,707
2         17,729
3            500
4            NaN
           ...  
212853    28,000
212854    13,000
212855       NaN
212856    30,000
212857    75,000
Name: 종전계약 보증금(만원), Length: 212858, dtype: object

In [ ]:
df['단지명'] == 'e편한세상신촌4단지' and df['종전계약 보증금(만원)'] <= 40000

SyntaxError: invalid decimal literal (2597041143.py, line 1)

In [83]:
df.columns

Index(['NO', '시군구', '번지', '본번', '부번', '단지명', '전월세구분', '전용면적(㎡)', '계약년월', '계약일',
       '보증금(만원)', '월세금(만원)', '층', '건축년도', '도로명', '계약기간', '계약구분', '갱신요구권 사용',
       '종전계약 보증금(만원)', '종전계약 월세(만원)', '주택유형'],
      dtype='object')

In [88]:
df.loc[(df['단지명'] == '예지보금자리74차')].filter(items=['NO', '시군구', '번지', '본번', '부번', '단지명', '전월세구분', '전용면적(㎡)', '계약년월', '계약일',
       '보증금(만원)', '월세금(만원)', '층'])

,NO,시군구,번지,본번,부번,단지명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),월세금(만원),층
29239,29240,서울특별시 양천구 신월동,480-4,480,4,예지보금자리74차,월세,55.69,202410,4,"10,000",60,4
160627,160628,서울특별시 양천구 신월동,480-4,480,4,예지보금자리74차,전세,44.80,202403,11,"20,000",0,7


In [91]:
df['보증금(만원)']

0         19,323
1         19,707
2         17,729
3            500
4          6,813
           ...  
212853    29,160
212854    13,000
212855         0
212856    40,000
212857    60,000
Name: 보증금(만원), Length: 212858, dtype: object

In [61]:
# CSVLoader 클래스를 사용하여 CSV 파일 로드
loader = CSVLoader(file_path='./data/아파트(전월세)_실거래가_2024.csv')  # CSV 파일 경로 지정

# CSV 파일에서 데이터 내용 로드
data = loader.load()  # 로드된 데이터가 data 변수에 저장

# 로드된 데이터 출력
data

[Document(metadata={'source': './data/아파트(전월세)_실거래가_2024.csv', 'row': 0}, page_content='NO: 1\n시군구: 서울특별시 용산구 한강로2가\n번지: 426\n본번: 426\n부번: 0\n단지명: 용산베르디움프렌즈\n전월세구분: 월세\n전용면적(㎡): 48.9018\n계약년월: 202412\n계약일: 6\n보증금(만원): 19,323\n월세금(만원): 34\n층: 15\n건축년도: 2021\n도로명: 백범로99길 40\n계약기간: 202503~202703\n계약구분: 갱신\n갱신요구권 사용: -\n종전계약 보증금(만원): 19,323\n종전계약 월세(만원): 33\n주택유형: 아파트'),
 Document(metadata={'source': './data/아파트(전월세)_실거래가_2024.csv', 'row': 1}, page_content='NO: 2\n시군구: 서울특별시 용산구 한강로2가\n번지: 426\n본번: 426\n부번: 0\n단지명: 용산베르디움프렌즈\n전월세구분: 월세\n전용면적(㎡): 49.9403\n계약년월: 202412\n계약일: 6\n보증금(만원): 19,707\n월세금(만원): 35\n층: 7\n건축년도: 2021\n도로명: 백범로99길 40\n계약기간: 202503~202703\n계약구분: 갱신\n갱신요구권 사용: -\n종전계약 보증금(만원): 19,707\n종전계약 월세(만원): 34\n주택유형: 아파트'),
 Document(metadata={'source': './data/아파트(전월세)_실거래가_2024.csv', 'row': 2}, page_content='NO: 3\n시군구: 서울특별시 용산구 한강로2가\n번지: 426\n본번: 426\n부번: 0\n단지명: 용산베르디움프렌즈\n전월세구분: 월세\n전용면적(㎡): 44.9009\n계약년월: 202412\n계약일: 6\n보증금(만원): 17,729\n월세금(만원): 32\n층: 9\n건축년도: 2021\n도로명:

In [20]:
data

[Document(metadata={'source': './data/TR222100060001_20230707.csv', 'row': 0}, page_content='sigungu_cd: 11110\nemdl_cd: 10700\ncll: 1\nmno: 0156\nsno: 0000\nadres: 서울특별시 종로구 적선동 156\nhus_tp: 오피스텔\ncomp_nm: 광화문 플래티넘\nbldg_year: 2003\nflr: 14\nxuar: 38.64\nctrt_yrmth: 202306\nctrt_day: 21\ntransct_type: 매매\ndlng_amount: 23000\ngrnte_amount: 0\nmthrnt_amount: 0\nnear_subw_nm: 경복궁(정부서울청사)역\nnear_subw_dist: 102.3'),
 Document(metadata={'source': './data/TR222100060001_20230707.csv', 'row': 1}, page_content='sigungu_cd: 11110\nemdl_cd: 11500\ncll: 1\nmno: 0009\nsno: 0000\nadres: 서울특별시 종로구 사직동 9\nhus_tp: 아파트\ncomp_nm: 광화문스페이스본(101동~105동)\nbldg_year: 2008\nflr: 13\nxuar: 108.55\nctrt_yrmth: 202306\nctrt_day: 17\ntransct_type: 월세\ndlng_amount: 0\ngrnte_amount: 60000\nmthrnt_amount: 120\nnear_subw_nm: 경복궁(정부서울청사)역\nnear_subw_dist: 236.2'),
 Document(metadata={'source': './data/TR222100060001_20230707.csv', 'row': 2}, page_content='sigungu_cd: 11110\nemdl_cd: 11500\ncll: 1\nmno: 0009\nsno: 0000\

In [62]:
# to db
db = FAISS.from_documents(
    documents=data,
    embedding=OpenAIEmbeddings(model='text-embedding-3-small')
)

In [63]:
# 생성한 FAISS 데이터베이스를 리트리버로 변환합니다.
retriever = db.as_retriever()

In [64]:
# RetrievalQA 체인을 생성합니다.
chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model_name='gpt-4o-mini'),  # 사용할 언어 모델 설정 (GPT-4o-mini 모델)
    chain_type='stuff',  # 체인 유형 설정 (여기서는 'stuff' 타입 사용)
    retriever=retriever,  # 이전에 설정한 리트리버 사용
)

In [81]:
chain.invoke('서울특별시 전세 중에서 보증금이 가장 저렴한 단지는 어디야?')

{'query': '서울특별시 전세 중에서 보증금이 가장 저렴한 단지는 어디야?',
 'result': '서울특별시 전세 중에서 보증금이 가장 저렴한 단지는 서울특별시 양천구 신월동에 위치한 "예지보금자리74차"로, 보증금은 20,000만원입니다.'}